In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from pandas import Timestamp
import csv
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [2]:
custom_date_parser=lambda dates: [ datetime.strptime(str(d), "%Y-%m-%d") for d in dates]  
# for datetime "%Y-%m-%d %H:%M:%S"
dfraw = pd.read_csv('data.csv', quoting=csv.QUOTE_NONNUMERIC,quotechar='"',sep = ',', header = 0, index_col = None)
dfraw["date_trans"]=pd.to_datetime(dfraw["date_trans"],format="%Y-%m-%d")
dfraw

,id,ref,date_trans,status,amount
0,1.0,INV-220001,2022-01-01,OPEN,35.94
1,2.0,INV-220002,2022-01-02,OPEN,99.99
2,3.0,INV-220003,2022-01-03,CANCELLED,13.00
3,4.0,INV-220003,2022-01-04,OPEN,NaN
4,5.0,INV-220005,2022-01-04,OPEN,204.20
5,6.0,INV-220006,2022-01-15,OPEN,-4.20
6,7.0,NaN,2022-01-06,NaN,0.00
7,8.0,INV-220007,2022-01-15,PENDING,50.40
8,9.0,NaN,NaT,ERROR,0.00
9,10.0,INV-220007,2022-01-15,PENDING,50.40


In [3]:
def dataframe_to_code(df):
    data = np.array2string(df.to_numpy(), separator=', ')
    data = data.replace(" nan", " float('nan')")
    data = data.replace(" NaT", " pd.NaT")

    cols = df.columns.tolist()
    return f"""df = pd.DataFrame({data}, columns={cols})"""
#print(dataframe_to_code(dfraw))
#use this to create the code for dataframes

In [4]:
def clean_cols(in_df,date_fillna="latest"):
    df=in_df.copy()
    dtypes = df.dtypes.to_dict()
    for col, typ in dtypes.items():
        if typ=="float" or typ=="int64":
            df[col].fillna(0,inplace=True)        
        elif is_datetime(df[col]):
            if date_fillna=="latest":
                val=max(df[col])
            elif date_fillna=="first":
                val=min(df[col])
            elif  isinstance(date_fillna, datetime):
                val=date_fillna
            df[col].fillna(val,inplace=True)
        elif typ=="object":
            df[col].fillna('',inplace=True)
    return df    
n=datetime.now()    
df=clean_cols(dfraw,date_fillna=n)
df

,id,ref,date_trans,status,amount
0,1.0,INV-220001,2022-01-01 00:00:00.000000,OPEN,35.94
1,2.0,INV-220002,2022-01-02 00:00:00.000000,OPEN,99.99
2,3.0,INV-220003,2022-01-03 00:00:00.000000,CANCELLED,13.00
3,4.0,INV-220003,2022-01-04 00:00:00.000000,OPEN,0.00
4,5.0,INV-220005,2022-01-04 00:00:00.000000,OPEN,204.20
5,6.0,INV-220006,2022-01-15 00:00:00.000000,OPEN,-4.20
6,7.0,,2022-01-06 00:00:00.000000,,0.00
7,8.0,INV-220007,2022-01-15 00:00:00.000000,PENDING,50.40
8,9.0,,2022-01-24 23:11:22.803175,ERROR,0.00
9,10.0,INV-220007,2022-01-15 00:00:00.000000,PENDING,50.40


In [5]:
df

,id,ref,date_trans,status,amount
0,1.0,INV-220001,2022-01-01 00:00:00.000000,OPEN,35.94
1,2.0,INV-220002,2022-01-02 00:00:00.000000,OPEN,99.99
2,3.0,INV-220003,2022-01-03 00:00:00.000000,CANCELLED,13.00
3,4.0,INV-220003,2022-01-04 00:00:00.000000,OPEN,0.00
4,5.0,INV-220005,2022-01-04 00:00:00.000000,OPEN,204.20
5,6.0,INV-220006,2022-01-15 00:00:00.000000,OPEN,-4.20
6,7.0,,2022-01-06 00:00:00.000000,,0.00
7,8.0,INV-220007,2022-01-15 00:00:00.000000,PENDING,50.40
8,9.0,,2022-01-24 23:11:22.803175,ERROR,0.00
9,10.0,INV-220007,2022-01-15 00:00:00.000000,PENDING,50.40


In [6]:
cols=["date_trans"]
for col in cols:
    unique =set([i.date() for i in set(df[pd.notna(df[col])][col])])
    fullrng=set(pd.date_range(min(unique),max(unique)-timedelta(days=1),freq='d').to_list())
    if unique.issubset(fullrng):
        print("Full sequence")
    else:
        diff=unique.difference(fullrng)
        print(len(diff)," missing in sequence, first ",min(len(diff),10),":",list(diff)[0:10])
        working_days=[wd for wd in diff if wd.weekday()<5]
        print(len(working_days), " missing working days", min(len(diff),10),":",list(diff)[0:10])
              

7  missing in sequence, first  7 : [datetime.date(2022, 1, 24), datetime.date(2022, 1, 15), datetime.date(2022, 1, 6), datetime.date(2022, 1, 4), datetime.date(2022, 1, 3), datetime.date(2022, 1, 2), datetime.date(2022, 1, 1)]
4  missing working days 7 : [datetime.date(2022, 1, 24), datetime.date(2022, 1, 15), datetime.date(2022, 1, 6), datetime.date(2022, 1, 4), datetime.date(2022, 1, 3), datetime.date(2022, 1, 2), datetime.date(2022, 1, 1)]


In [9]:
def create_calendar(start='2000-01-01', end='2050-12-31'):
    df = pd.DataFrame({"date": pd.date_range(start, end)})
    
    df["day"] = df.date.dt.day
    df["month"] = df['date'].dt.month
    df["week"] = df['date'].dt.week
    df["quarter"] = df.date.dt.quarter
    df["year"] = df.date.dt.year
    df["weekend"]=df.date.dt.weekday>=5
    df["weekday"]=df.date.dt.weekday
    df["weekday_name"]=df.date.dt.day_name()
    df["weekday_name_short"]=df.date.dt.day_name().str[0:3]
    df["day_of_year"]=df.date.dt.dayofyear

    
    df.insert(0, 'date_id', (df.year.astype(str) + df.month.astype(str).str.zfill(2) + df.day.astype(str).str.zfill(2)).astype(int))
    df.insert(2, 'yyyymm', (df.year.astype(str) + df.month.astype(str).str.zfill(2)))
    df.insert(3, 'yyyyqq', (df.year.astype(str) + df.quarter.astype(str).str.zfill(2)))
    df.insert(4, 'yyyyww', (df.year.astype(str) + df.week.astype(str).str.zfill(2)))
    
    return df


create_calendar(min(df["date_trans"]),max(df["date_trans"]))


,date_id,date,yyyymm,yyyyqq,yyyyww,day,month,week,quarter,year,weekend,weekday,weekday_name,weekday_name_short,day_of_year
0,20220101,2022-01-01,202201,202201,202252,1,1,52,1,2022,True,5,Saturday,Sat,1
1,20220102,2022-01-02,202201,202201,202252,2,1,52,1,2022,True,6,Sunday,Sun,2
2,20220103,2022-01-03,202201,202201,202201,3,1,1,1,2022,False,0,Monday,Mon,3
3,20220104,2022-01-04,202201,202201,202201,4,1,1,1,2022,False,1,Tuesday,Tue,4
4,20220105,2022-01-05,202201,202201,202201,5,1,1,1,2022,False,2,Wednesday,Wed,5
5,20220106,2022-01-06,202201,202201,202201,6,1,1,1,2022,False,3,Thursday,Thu,6
6,20220107,2022-01-07,202201,202201,202201,7,1,1,1,2022,False,4,Friday,Fri,7
7,20220108,2022-01-08,202201,202201,202201,8,1,1,1,2022,True,5,Saturday,Sat,8
8,20220109,2022-01-09,202201,202201,202201,9,1,1,1,2022,True,6,Sunday,Sun,9
9,20220110,2022-01-10,202201,202201,202202,10,1,2,1,2022,False,0,Monday,Mon,10
